In [ ]:
# Instalar dependências
!pip install unidecode google-api-python-client gspread rapidfuzz beautifulsoup4 lxml --upgrade --quiet
!pip install requests==2.32.4 --quiet

print("Dependências instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 104.7 MB/s eta 0:00:00
Dependências instaladas com sucesso!


In [ ]:
# -*- coding: utf-8 -*-

# Importar bibliotecas
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict
from bs4 import BeautifulSoup
from rapidfuzz import fuzz, process as rf_process
from io import BytesIO
import pandas as pd
import numpy as np
import random, time, requests, re, os
import unicodedata
from unidecode import unidecode

# Autenticação e cliente gspread
import google.auth
import gspread
from google.colab import auth
from gspread.client import Client

print("Bibliotecas instaladas com sucesso!")

Bibliotecas instaladas com sucesso!


In [ ]:
# @title **ETAPA 2.1. Geração do `df_mapa_RFEPT` obtido via Web Scraping**
# para mesclagem visando a complementação de informações de `df_rede_federal` (ETAPA 1)

# --- CONSTANTES GLOBAIS ---

# URL base do portal do MEC para a Rede Federal
URL_BASE = "https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/{uf}"

# Lista das 27 Unidades Federativas do Brasil
UFS = [
    "acre","alagoas","amapa","amazonas","bahia","ceara","distrito-federal","espirito-santo",
    "goias","maranhao","mato-grosso","mato-grosso-do-sul","minas-gerais","para","paraiba",
    "parana","pernambuco","piaui","rio-de-janeiro","rio-grande-do-norte","rio-grande-do-sul",
    "rondonia","roraima","santa-catarina","sao-paulo","sergipe","tocantins",
]

# Cabeçalho da requisição para simular um navegador e evitar bloqueios
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# SESSION = requests.Session()
# SESSION.headers.update(HEADERS)

# Textos que indicam um campo vazio ou não preenchido
PLACEHOLDER_TEXTS = [
    'não especificado', 'não informado', 'informação indisponível',
    'não há', 'não se aplica'
]

# Palavras-chave para identificar o início de um bloco de instituição (Reitoria)
INSTITUTION_KEYWORDS = ['INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA',
                        'CEFET',
                        'UNIVERSIDADE TECNOLÓGICA FEDERAL'
                        ]

###  **Função `scrape_all_ufs`**

Esta função orquestra todo o processo de _web scraping_, itera sobre a lista de UFs, gerencia as requisições HTTP e delega a análise do conteúdo para funções auxiliares.

In [ ]:
def scrape_all_ufs():
    """
    Orquestra o processo de scraping para todas as 27 Unidades Federativas.

    Itera sobre a lista de UFS, faz as requisições HTTP, trata possíveis erros
    de conexão e agrega os resultados de cada página em uma lista única.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário contém os dados
              de uma unidade (Reitoria ou Campus).
    """
    all_data = []
    log_summary = []

    print("Iniciando processo de extração de dados da Rede Federal...")

    for uf in UFS:
        url = URL_BASE.format(uf=uf.lower())
        print(f"Processando UF: {uf.upper()} | URL: {url}")

        try:
            # Realiza a requisição HTTP com um timeout de 20 segundos
            response = requests.get(url, headers=HEADERS, timeout=20)
            # Levanta uma exceção para códigos de status de erro (4xx ou 5xx)
            response.raise_for_status()

            # Passa o conteúdo HTML para a função de parsing
            # uf_data = parse_uf_page(response.content, uf.upper())
            uf_data = parse_uf_page(response.content, uf)

            if uf_data:
                all_data.extend(uf_data)
                log_summary.append({
                    'UF': uf.upper(),
                    'Status': 'Success',
                    'Units Found': len(uf_data)})
                print(f"  -> Sucesso: {len(uf_data)} unidades encontradas.")
            else:
                log_summary.append({
                    'UF': uf.upper(),
                    'Status': 'Failed - No Data Found',
                    'Institutes Found': 0,
                    'Campuses Found': 0
                })
                print("  -> Alerta: Página acessada, mas nenhum dado estruturado foi encontrado.")

        except requests.exceptions.RequestException as e:
        # Captura erros de rede, timeout, ou status HTTP de erro
            log_summary.append({
                'UF': uf.upper(),
                'Status': f'Failed - {type(e).__name__}',
                'Units Found': 0})
            print(f"  -> ERRO: Falha ao acessar a URL. Motivo: {e}")

        # Pausa de 3 segundos entre as requisições para não sobrecarregar o servidor
        time.sleep(3)

    print("\nProcesso de extração finalizado.")
    return all_data, log_summary

### **Função `parse_uf_page` (análise e extração)**
Esta função é o core do script, recebe o conteúdo HTML de uma página, aplica a lógica de _parsing_ com estado para identificar Reitorias e Campi, e utiliza funções auxiliares para extrair os dados de forma defensiva.

In [ ]:
def extrair_informacoes(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml')

        # Dicionário para armazenar os dados
        dados = {'Endereço': None, 'CEP': None, 'Telefone': None, 'E-mail': None, 'Site': None}

        # --- Correção 1: Seletores de Container Flexíveis ---
        seletores_container = [
            'div.content-col-content',     # Padrão antigo
            'article#parent-fieldname-text', # Padrão Plone
            'div#content-core',            # Padrão Plone
            'div#portal-column-content',   # Padrão Plone
            'div.entry-content',           # Padrão Wordpress
            'div#content',
            'main#main-content',
            'main',
            'article',
            'body' # Em último caso, usa o corpo todo
        ]

        container = None
        for seletor in seletores_container:
            container = soup.select_one(seletor)
            if container:
                # print(f"DEBUG: Container encontrado com o seletor: {seletor} em {url}")
                break

        if not container:
            print(f"ALERTA: Nenhum container encontrado para a URL: {url}")
            return dados # Retorna dados vazios

        # --- Correção 2: Extração baseada em Pistas ---

        # 1. Extrair Telefone
        pistas_tel = ['fa-phone', 'icon-phone', 'Telefone:', 'Tel.:', 'Fone:']
        dados['Telefone'] = buscar_info_por_pista(container, pistas_tel, 'telefone', url)

        # 2. Extrair E-mail
        pistas_email = ['fa-envelope', 'icon-envelope', 'E-mail:', 'Email:', 'Correio Eletrônico:']
        dados['E-mail'] = buscar_info_por_pista(container, pistas_email, 'email', url)

        # 3. Extrair Endereço
        pistas_end = ['fa-map-marker', 'icon-map-marker', 'Endereço:', 'Localização:']
        dados['Endereço'] = buscar_info_por_pista(container, pistas_end, 'endereco', url)

        # 4. Extrair CEP (Muitas vezes vem junto com o endereço ou tem sua própria pista)
        pistas_cep = ['CEP:']
        dados['CEP'] = buscar_info_por_pista(container, pistas_cep, 'cep', url)

        # 5. Extrair Site (Lógica um pouco diferente)
        pistas_site = ['fa-globe', 'icon-globe', 'Site:', 'Página:', 'Portal:']
        try:
            el_site = None
            for pista in pistas_site:
                elemento_encontrado = None
                if pista.startswith('fa-') or pista.startswith('icon-'):
                    elemento_encontrado = container.select_one(f"[class*='{pista}']")
                else:
                    elemento_encontrado = container.find(string=re.compile(pista, re.IGNORECASE))

                if elemento_encontrado:
                    # Procura o link '<a>' no elemento "pai"
                    el_pai = elemento_encontrado.parent
                    link_site = el_pai.find('a', href=re.compile(r'http'))
                    if link_site and 'mailto:' not in link_site['href']:
                        dados['Site'] = link_site['href'].strip()
                        break

            # Fallback (Plano B) para Site: Procurar o primeiro link 'http'
            # que pareça ser o site principal (evitar links de redes sociais)
            if not dados['Site']:
                todos_links = container.find_all('a', href=re.compile(r'http'))
                for link in todos_links:
                     href = link['href']
                     if 'mailto:' not in href and 'facebook.com' not in href and \
                        'instagram.com' not in href and 'youtube.com' not in href and \
                        url not in href: # Evita links para a própria página

                         dados['Site'] = href.strip()
                         break # Pega o primeiro link externo válido
        except Exception:
            pass # Falha silenciosa na busca do site


        # --- PLANO B (Fallback) ---
        # Se a busca por pistas falhou, tentamos o método antigo (Regex no texto todo)
        # Isso garante que não perdemos dados que o script antigo já pegava

        full_text = container.get_text(strip=True, separator=' ')

        if not dados['Telefone']:
            match = re.search(r'((?:\(?\b\d{2}\)?\s?)?\d{4,5}[-.\s]?\d{4})', full_text)
            if match: dados['Telefone'] = match.group(1).strip()

        if not dados['E-mail']:
            match = re.search(r'([\w\.-]+@[\w\.-]+\.\w+)', full_text, re.IGNORECASE)
            if match: dados['E-mail'] = match.group(1).strip()

        # O CEP pode estar no endereço, então buscamos ele antes de limpar
        if not dados['CEP']:
            match_cep_fallback = re.search(r'(\b\d{5}-?\d{3}\b)', full_text)
            if match_cep_fallback:
                dados['CEP'] = match_cep_fallback.group(1).strip()
                # Se o endereço foi pego junto, vamos usá-lo
                if not dados['Endereço']:
                     # Tenta pegar o texto ao redor do CEP (Contexto)
                    texto_proximo = full_text[max(0, match_cep_fallback.start()-100) : match_cep_fallback.end()+50]
                    # Tenta limpar o contexto
                    if 'Endereço:' in texto_proximo:
                         texto_proximo = texto_proximo.split('Endereço:')[1]
                    dados['Endereço'] = texto_proximo.strip(' ,-')


        # --- Limpeza Final ---
        # Se o CEP foi pego (pela pista ou fallback) e também está dentro do Endereço
        if dados['Endereço'] and dados['CEP'] and dados['CEP'] in dados['Endereço']:
            # Remove o CEP e o texto "CEP:" do endereço
            dados['Endereço'] = dados['Endereço'].replace(dados['CEP'], '').strip()
            dados['Endereço'] = re.sub(r'CEP:?', '', dados['Endereço'], flags=re.IGNORECASE).strip(' ,-')

        return dados

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return {'Endereço': None, 'CEP': None, 'Telefone': None, 'E-mail': None, 'Site': None}

In [ ]:
def parse_uf_page(html_content, uf_sigla):
    """
    Analisa o conteúdo HTML de uma página de UF e extrai os dados das instituições.
    Primeiro, localiza os títulos e depois processa o bloco pai correspondente.

    Implementa uma lógica 'stateful' para associar corretamente os campi às suas
    respectivas reitorias, especialmente em páginas com múltiplos institutos.

    Args:
        html_content (bytes): O conteúdo HTML bruto da página.
        uf_sigla (str): A sigla da UF sendo processada (ex: 'SP').

    Returns:
        list: Uma lista de dicionários com os dados das unidades encontradas na página.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    content_area = soup.find(id='content-core')
    if not content_area:
        return

    all_units_data = []
    current_institute_info = {}

    # Abordagem mais robusta: encontrar todos os títulos e iterar sobre eles
    titles = content_area.find_all(['b', 'strong'])

    for title in titles:
        text = title.get_text(strip=True).upper()

        # Encontra o contêiner do bloco (ex: <p>, <td>, <div>)
        parent_block = title.find_parent(['p', 'td', 'div'])
        if not parent_block:
            continue

        # Verifica se é um bloco de Instituição
        if any(keyword in text for keyword in INSTITUTION_KEYWORDS):
            institution_data = parse_institution_block(parent_block, uf_sigla)
            if institution_data:
                current_institute_info = {
                    'Nome_IF': institution_data.get('Nome_IF'),
                    'Sigla_IF': institution_data.get('Sigla_IF'),
                    'Site': institution_data.get('Site')
                }
                all_units_data.append(institution_data)

        # Verifica se é um bloco de Campus
        elif text.startswith("CAMPUS"):
            if not current_institute_info:
                continue  # Evita campus órfão

            campus_data = parse_campus_block(parent_block, uf_sigla)
            if campus_data:
                # Combina os dados do instituto com os dados do campus
                full_data = {**current_institute_info, **campus_data}
                all_units_data.append(full_data)

    return all_units_data

In [ ]:
def buscar_info_por_pista(container, pistas, tipo_info, debug_url=''):
    """
    Procura por elementos que contenham 'pistas' (texto ou classes CSS)
    e extrai a informação relevante (próximo texto ou atributo).

    pistas: lista de strings para procurar (ex: ['fa-phone', 'Telefone:'])
    tipo_info: 'email', 'telefone', 'cep', 'site', 'endereco'
    """

    try:
        # 1. Tenta procurar por texto visível (ex: "Telefone:")
        # Usamos 're.compile' para ignorar maiúsculas/minúsculas (case-insensitive)
        elementos_pista = container.find_all(string=re.compile(r'|'.join(pistas), re.IGNORECASE))

        if elementos_pista:
            for el in elementos_pista:
                # A informação pode estar no texto do elemento "pai"
                texto_pai = el.parent.get_text(strip=True, separator=' ')

                # Tenta extrair usando regex específico do tipo
                if tipo_info == 'telefone':
                    match = re.search(r'((?:\(?\b\d{2}\)?\s?)?\d{4,5}[-.\s]?\d{4})', texto_pai)
                    if match: return match.group(1).strip()

                if tipo_info == 'email':
                    # O email pode estar num link 'mailto:' próximo
                    link_email = el.parent.find('a', href=re.compile(r'mailto:'))
                    if link_email:
                        return link_email['href'].replace('mailto:', '').strip()
                    match = re.search(r'([\w\.-]+@[\w\.-]+\.\w+)', texto_pai, re.IGNORECASE)
                    if match: return match.group(1).strip()

                if tipo_info == 'cep':
                    match = re.search(r'(\b\d{5}-?\d{3}\b)', texto_pai)
                    if match: return match.group(1).strip()

                if tipo_info == 'endereco':
                    # Remove a pista (ex: "Endereço:") do texto
                    texto_limpo = re.sub(r'|'.join(pistas), '', texto_pai, flags=re.IGNORECASE).strip(' :')
                    # Verifica se sobrou texto útil (evita pegar só um "Endereço:")
                    if len(texto_limpo) > 10:
                        return texto_limpo

        # 2. Se não achou por texto, tenta por classes CSS (ícones)
        classes_css = [p for p in pistas if p.startswith('fa-') or p.startswith('icon-')]
        if classes_css:
            # Constrói um seletor CSS (ex: "[class*='fa-phone'], [class*='icon-phone']")
            seletor_icone = ', '.join([f"[class*='{c}']" for c in classes_css])
            el_icone = container.select_one(seletor_icone)

            if el_icone:
                # A informação geralmente está no texto do elemento "pai"
                elemento_pai = el_icone.parent
                # Às vezes, o elemento pai é o próprio link (ex: <a><i class="fa-phone"></i> (44)...</a>)
                # Vamos subir até 2 níveis se necessário
                if len(elemento_pai.get_text(strip=True)) < 8 and elemento_pai.parent:
                    elemento_pai = elemento_pai.parent

                texto_pai_icone = elemento_pai.get_text(strip=True, separator=' ')

                if tipo_info == 'telefone':
                    match = re.search(r'((?:\(?\b\d{2}\)?\s?)?\d{4,5}[-.\s]?\d{4})', texto_pai_icone)
                    if match: return match.group(1).strip()

                if tipo_info == 'email':
                    link_email = elemento_pai.find('a', href=re.compile(r'mailto:'))
                    if link_email:
                        return link_email['href'].replace('mailto:', '').strip()
                    match = re.search(r'([\w\.-]+@[\w\.-]+\.\w+)', texto_pai_icone, re.IGNORECASE)
                    if match: return match.group(1).strip()

                if tipo_info == 'cep':
                    match = re.search(r'(\b\d{5}-?\d{3}\b)', texto_pai_icone)
                    if match: return match.group(1).strip()

                if tipo_info == 'endereco':
                     # Remove a pista (ex: "Endereço:") do texto
                    texto_limpo = re.sub(r'|'.join(pistas), '', texto_pai_icone, flags=re.IGNORECASE).strip(' :')
                    if len(texto_limpo) > 10:
                        return texto_limpo

    except Exception as e:
        # print(f"Erro ao buscar '{tipo_info}' em {debug_url}: {e}")
        pass # Continua silenciosamente

    return None

In [ ]:
def extract_field(block, label):
    """
    Função de extração defensiva genérica. Busca um rótulo e extrai o texto subsequente.

    Args:
        block (bs4.element.Tag): O objeto BeautifulSoup do bloco de conteúdo.
        label (str): O rótulo de texto a ser procurado (ex: 'Telefone:').

    Returns:
        str or None: O valor limpo do campo, ou None se não for encontrado ou for um placeholder.
    """
    try:
        block_text = block.get_text('\n', strip=True)  # Define block_text here
        # Usa regex para encontrar o rótulo de forma flexível (ignorando maiúsculas/minúsculas)
        # e captura o texto até o próximo rótulo ou final da linha.
        # Usa raw string (r"...") para evitar SyntaxWarning com '\s'

        # pattern = re.compile(rf"{re.escape(label)}[:\s]*([^\n\r]+)", re.IGNORECASE)
        pattern = re.compile(f"{re.escape(label)}[:\\s]*([^\\n\\r]+)", re.IGNORECASE)
        # match = pattern.search(block.get_text('\n', strip=True))
        match = pattern.search(block_text)
        if match:
            value = match.group(1).strip()
            # Verifica se o valor extraído não é um placeholder
            if not any(ph.lower() in value.lower() for ph in PLACEHOLDER_TEXTS):
                return value
    except Exception:
        pass
    return None


def extract_email(block):
    """Função aprimorada para extrair e-mail, procurando por links 'mailto:'."""
    try:
        # Prioriza a busca por links mailto:, que é a forma mais comum e confiável
        email_link = block.find('a', href=re.compile(r'^mailto:'))
        if email_link:
            return email_link.get_text(strip=True)

        # Se não encontrar, tenta a extração baseada em texto como fallback
        return extract_field(block, 'E-mail')
    except Exception:
        return None

def parse_location_string(block_text, uf_name):
    """Analisa o texto para extrair Município, UF e CEP com base em padrões comuns."""
    # Converte o nome completo da UF (ex: "sao-paulo") para a sigla (ex: "SP") para fallback
    uf_sigla = ''.join([word for word in uf_name.split('-')]).upper()
    if len(uf_sigla) > 2: # Heurística simples para siglas compostas
        uf_sigla = ''.join([word for word in uf_name.upper().split('-')])

    municipio, uf, cep = None, uf_sigla, None

    # Padrão 1: "Cidade/UF - CEP: 12345-678"
    match = re.search(r'([A-Za-z\s\'-À-ú]+)\s*/\s*([A-Z]{2})\s*-\s*CEP:\s*(\d{5}-\d{3})', block_text)
    if match:
        municipio = match.group(1).strip().rstrip('., ')
        uf = match.group(2).strip()
        cep = match.group(3).strip()
    else:
        # Padrão 2 (fallback): Tenta extrair apenas o CEP se o padrão completo falhar
        cep_match = re.search(r'(\d{5}-\d{3})', block_text)
        if cep_match:
            cep = cep_match.group(1)

    return {'Município': municipio, 'UF': uf, 'CEP': cep}

# -----------------------------------
# --- FUNÇÕES DE PARSING DE BLOCO ---
# -----------------------------------

def parse_institution_block(block, uf_name):
    """Extrai e estrutura os dados de um bloco de Instituição (Reitoria)."""
    block_text = block.get_text('\n', strip=True)

    # Extrai o nome completo da instituição
    title_tag = block.find(['b', 'strong'])
    nome_if = title_tag.get_text(strip=True) if title_tag else ""

    # Extrai a sigla do nome completo (ex: (IFSP))
    sigla_match = re.search(r'\((.*?)\)', nome_if)
    sigla_if = sigla_match.group(1) if sigla_match else None

    # Limpa o nome do IF removendo a sigla
    nome_if_clean = re.sub(r'\s*\((.*?)\)', '', nome_if).strip()

    location_parts = parse_location_string(block_text, uf_name)

    return {
        'Sigla_IF': sigla_if,
        'Nome_IF': nome_if_clean,
        'Campus_IF': 'Reitoria',
        'Município': location_parts['Município'],
        'UF': location_parts['UF'],
        'Endereço': extract_field(block, 'Endereço'),
        'CEP': location_parts['CEP'] or extract_field(block, 'CEP'),
        'Telefone': extract_field(block, 'Telefone'),
        'E-mail': extract_email(block),
        'Site': extract_field(block, 'Site')
    }

def parse_campus_block(block, uf_name):
    """Extrai e estrutura os dados de um bloco de Campus."""
    block_text = block.get_text('\n', strip=True)

    title_tag = block.find(['b', 'strong'])
    campus_name_raw = title_tag.get_text(strip=True) if title_tag else ""

    # Limpa e padroniza o nome do campus
    campus_name_clean = re.sub(r'^(CAMPUS|AVANÇADO)\s*', '', campus_name_raw, flags=re.IGNORECASE).strip()
    campus_name = campus_name_clean.title() # Aplica Title Case

    location_parts = parse_location_string(block_text, uf_name)

    return {
        'Campus_IF': campus_name,
        'Município': location_parts['Município'],
        'UF': location_parts['UF'],
        'Endereço': extract_field(block, 'Endereço'),
        'CEP': location_parts['CEP'] or extract_field(block, 'CEP'),
        'Telefone': extract_field(block, 'Telefone'),
        'E-mail': extract_email(block),
    }


def parse_address_string(address_str, uf_sigla):
    """
    Analisa a string de endereço para extrair Endereço, Município, UF e CEP.

    Args:
        address_str (str): A string completa do endereço.
        uf_sigla (str): A sigla da UF da página, usada como fallback.

    Returns:
        dict: Um dicionário contendo 'Endereço', 'Município', 'UF' e 'CEP'.
    """
    if not address_str:
        return {'Endereço': None, 'Município': None, 'UF': uf_sigla, 'CEP': None}

    # Tenta extrair o CEP usando regex
    cep_match = re.search(r'(\d{5}-\d{3})', address_str)
    cep = cep_match.group(1) if cep_match else None

    # Tenta extrair Município/UF
    # Ex: "Araraquara/SP" ou "São Lourenço do Oeste/SC"
    municipio_uf_match = re.search(r'([A-Za-z\s\'-]+)\s*/\s*([A-Z]{2})', address_str)
    if municipio_uf_match:
        municipio = municipio_uf_match.group(1).strip().rstrip('. ,')
        uf = municipio_uf_match.group(2).strip()
    else:
        municipio = None
        uf = uf_sigla # Usa a UF da página como fallback

    return {'Endereço': address_str, 'Município': municipio, 'UF': uf, 'CEP': cep}

def parse_reitoria_block(block, uf_sigla):
    """
    Extrai e estrutura os dados de um bloco de Reitoria.
    """
    block_text = block.get_text('\n', strip=True) # Define block_text here
    # Tenta encontrar a tag strong, se não encontrar, pega o texto do block e junta as linhas
    nome_if_raw = block.find('strong').get_text(strip=True) if block.find('strong') else " ".join(block.get_text(strip=True).split('\n'))
    nome_if = nome_if_raw.strip()

    # Extrai a sigla do nome completo (ex: (IFSP))
    sigla_match = re.search(r'\((.*?)\)', nome_if)
    sigla_if = sigla_match.group(1) if sigla_match else None

    address_str = extract_field(block, 'Endereço')
    address_parts = parse_address_string(address_str, uf_sigla)

    return {
        'Sigla_IF': sigla_if,
        'Nome_IF': nome_if,
        'Campus_IF': 'Reitoria',
        'Município': address_parts['Município'],
        'UF': address_parts['UF'],
        'Endereço': address_parts['Endereço'],
        'CEP': address_parts['CEP'] or extract_field(block, 'CEP'),
        'Telefone': extract_field(block, 'Telefone'),
        'E-mail': extract_field(block, 'E-mail'),
        'Site': extract_field(block, 'Site')
    }

### **Montagem e Refinamento do DataFrame**
O bloco final de código executa o processo de scraping, converte os resultados em um DataFrame pandas e realiza as últimas etapas de limpeza e formatação.

In [ ]:
# --- EXECUÇÃO PRINCIPAL ---

# 1. Executa o motor de scraping para obter os dados brutos e o log
scraped_data, run_log = scrape_all_ufs()

# 2. Converte o log em um DataFrame para análise
df_log = pd.DataFrame(run_log)

# 3. Converte a lista de dicionários em um DataFrame
if scraped_data:
    df_mapa_RFEPT = pd.DataFrame(scraped_data)

    # --- LIMPEZA E REFINAMENTO DO DATAFRAME ---

    # Define a ordem desejada para as colunas
    desired_columns = [
        'Sigla_IF',
        'Nome_IF',
        'Campus_IF',
        'Município',
        'UF',
        'Endereço',
        'CEP',
        'Telefone',
        'E-mail',
        'Site'
    ]
    # Reorganiza as colunas, mantendo apenas as que existem no DataFrame
    # df_mapa_RFEPT = df_mapa_RFEPT.reindex(columns=[col for col in desired_columns if col in df_mapa_RFEPT.columns])
    df_mapa_RFEPT = df_mapa_RFEPT.reindex(columns=desired_columns)

    # Limpa possíveis caracteres indesejados nas colunas de texto
    text_cols_to_clean = ['Sigla_IF', 'Nome_IF', 'Campus_IF', 'Município', 'UF', 'Endereço', 'CEP', 'Telefone', 'E-mail', 'Site']
    for col in text_cols_to_clean:
        if col in df_mapa_RFEPT.columns and df_mapa_RFEPT[col].dtype == 'object':
            df_mapa_RFEPT[col] = df_mapa_RFEPT[col].astype(str).str.strip().replace('', pd.NA)

    # Limpa espaços em branco extras
    for col in text_cols_to_clean:
        if col in df_mapa_RFEPT.columns:
            df_mapa_RFEPT[col] = df_mapa_RFEPT[col].astype(str).str.strip()


    # Normaliza números de telefone, removendo caracteres não numéricos
    if 'Telefone' in df_mapa_RFEPT.columns:
        # Garante que a coluna é string antes de aplicar regex
        df_mapa_RFEPT['Telefone'] = df_mapa_RFEPT['Telefone'].astype(str).str.replace(r'[^\d]', '', regex=True)
        # Substitui strings vazias resultantes da limpeza por NA
        df_mapa_RFEPT['Telefone'] = df_mapa_RFEPT['Telefone'].replace('', pd.NA)


    # Exibe o log de execução
    print("\n--- Resumo da Execução do Scraping ---")
    print(df_log.to_string())

    # Exibe informações e as primeiras linhas do DataFrame final
    print("\n--- Informações do DataFrame 'df_mapa_RFEPT' ---")
    df_mapa_RFEPT.info()

    print("\n--- Amostra dos Dados Coletados (df_mapa_RFEPT) ---")
    display(df_mapa_RFEPT.head(10))

else:
    print("\nNenhum dado foi extraído. O DataFrame 'df_mapa_RFEPT' não pôde ser criado.")
    print("\n--- Resumo da Execução do Scraping ---")
    print(df_log.to_string())

Iniciando processo de extração de dados da Rede Federal...
Processando UF: ACRE | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/acre
  -> Sucesso: 7 unidades encontradas.
Processando UF: ALAGOAS | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/alagoas
  -> Sucesso: 17 unidades encontradas.
Processando UF: AMAPA | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/amapa
  -> Sucesso: 7 unidades encontradas.
Processando UF: AMAZONAS | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/amazonas
  -> Sucesso: 18 unidades encontradas.
Processando UF: BAHIA | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/bahia
  -> Sucesso: 41 unidades encontradas.
Processando UF: CEARA | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/ceara
  -> Sucesso: 34 unidades encontradas.
Processando UF: DISTRITO-FEDERAL | URL: https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/distrito-federal
  -> Sucesso: 11 unidades encontradas.
Processand

,Sigla_IF,Nome_IF,Campus_IF,Município,UF,Endereço,CEP,Telefone,E-mail,Site
0,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Reitoria,None,ACRE,None,None,<NA>,None,None
1,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Cruzeiro Do Sul,None,ACRE,"Rua Paraná, 25 de agosto. Cruzeiro do Sul, AC.",6998000.,4999820966,None,None
2,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Rio Branco,None,ACRE,"Av. Brasil, Xavier Maia. Rio Branco, AC. CEP: ...",69914-610,6821065910,None,None
3,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Rio Branco Baixada Do Sol,None,ACRE,"Rua Rio Grande do Sul, Aeroporto Velho. Rio Br...",69911-030,6832246816,reitoria@ifac.edu.br,None
4,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Sena Madureira,None,ACRE,"TRAVESSA GUILHERME, Pista. Sena Madureira, AC....",69940-970,6836122797,None,None
5,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Tarauacá,None,ACRE,"Rua Coronel Juvêncio de Menezes, Rua João Pess...",69970-000,6834621709,None,None
6,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Xapuri,None,ACRE,"Rua Seis de Agosto, Centro. Xapuri, AC. CEP: 6...",69930-000,<NA>,None,None
7,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Reitoria,None,ALAGOAS,None,None,<NA>,None,None
8,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Arapiraca,None,ALAGOAS,"Rodovia AL-110, Próximo Rotatória Polícia Rodo...",57302-045,8231941150,None,None
9,None,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Avançado Maceió Benedito Bentes,None,ALAGOAS,"Avenida Benedito Bentes, Benedito Bentes. Mace...",57084-651,8221266230,None,None


In [ ]:
# @title **ETAPA 2.2. Exportar DataFrame `df_mapa_RFEPT`para Google Sheets por ID**

# ======================== Configurações da Planilha ========================
# ID da planilha
ID_PLANILHA = '19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM'
# Nome da aba, se não existir, será criada.
NOME_ABA = 'Web_Scraping_Mapa_RFEPCT'
# ==============================================================

# 1. Autenticação do Google Colab
print("Autenticando-se no Google...")
# Inicia autenticação no Google
auth.authenticate_user()
print("Autenticação concluída.")

# 2. Inicializar o cliente gspread
# Utilizar as credenciais padrão estabelecidas pelo auth.authenticate_user()
credentials, project = google.auth.default()
gc = Client(auth=credentials)

try:
    # 3. Abrir a planilha usando o ID (Chave)
    sh = gc.open_by_key(ID_PLANILHA)
    print(f"Planilha ID: '{ID_PLANILHA}' acessada com sucesso.")

    # 4. Selecionar ou criar a aba (worksheet)
    try:
        # Tenta selecionar a aba existente
        worksheet = sh.worksheet(NOME_ABA)
        # Limpa o conteúdo existente para substituí-lo
        worksheet.clear()
        print(f"Aba '{NOME_ABA}' limpa e pronta para receber os novos dados.")
    except gspread.exceptions.WorksheetNotFound:
        # Se não encontrar, adiciona uma nova aba
        # Adicionamos um número grande de linhas/colunas para garantir espaço
        worksheet = sh.add_worksheet(title=NOME_ABA, rows="1000", cols="15")
        print(f"Aba '{NOME_ABA}' criada.")


    # 5. Converter o DataFrame para uma lista de listas (formato que o gspread usa)
    # Inclui o cabeçalho (nomes das colunas)
    # Usar .tolist() (sem underscore) para converter o array NumPy
    # Certifica-se que df_rede_federal está definido e não vazio
    if 'df_mapa_RFEPT' in locals() and not df_mapa_RFEPT.empty:
        # Replace pd.NA with None for JSON serialization
        df_mapa_RFEPT_cleaned = df_mapa_RFEPT.replace({pd.NA: None})
        dados_para_sheet = [df_mapa_RFEPT_cleaned.columns.tolist()] + df_mapa_RFEPT_cleaned.values.tolist()
    else:
        raise ValueError("DataFrame 'df_mapa_RFEPT' não encontrado ou está vazio.")


    # 6. Exportar os dados (atualiza toda a faixa a partir de A1)
    # Usar argumentos nomeados para evitar o DeprecationWarning
    worksheet.update(values=dados_para_sheet, range_name='A1')

    # 7. Exibir o link
    print("-" * 50)
    print("Exportação concluída com sucesso!")
    print(f"Acesse a planilha aqui: {sh.url}")
    print("-" * 50)

except gspread.exceptions.SpreadsheetNotFound:
    print("-" * 50)
    print(f"ERRO: Planilha com ID '{ID_PLANILHA}' não encontrada.")
    print("Verifique se o ID está correto ou se você possui acesso a ela.")
    print("-" * 50)
except gspread.exceptions.APIError as e:
    print("-" * 50)
    print("ERRO DE API (Permissão):")
    print("Verifique se a conta Google usada na autenticação tem permissão de 'Editor' na planilha.")
    print(f"Detalhes do erro: {e}")
    print("-" * 50)
except Exception as e:
    # Captura outros erros, incluindo problemas de permissão
    print(f"Ocorreu um erro inesperado durante a exportação: {e}")

Autenticando-se no Google...
Autenticação concluída.
Planilha ID: '19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM' acessada com sucesso.
Aba 'Web_Scraping_Mapa_RFEPCT' limpa e pronta para receber os novos dados.
--------------------------------------------------
Exportação concluída com sucesso!
Acesse a planilha aqui: https://docs.google.com/spreadsheets/d/19JkJDxBOuuoED-IMt07ofEKWy5fxnIOf5BbZikM2HPM
--------------------------------------------------
